# 定数変更

In [1]:
#!sed --help

!sed -i s/"'gna_'  : 16"/"'gna_'  : 4"/ ./elecpy/cell/luorudy/const.py 
!sed -i s/"'gacai_' : 0.01"/"'gacai_' : 0.01*0.68"/ ./elecpy/cell/luorudy/const.py 
!sed -i s/"'gacao_' : 0.341"/"'gacao_' : 0.341*0.68"/ ./elecpy/cell/luorudy/const.py 



#!cat elecpy/cell/luorudy/const.py 
!cat elecpy/cell/luorudy/const.py  | grep gna_

!cat elecpy/cell/luorudy/const.py  | grep gacai_
!cat elecpy/cell/luorudy/const.py  | grep gacao_

#!cat elecpy/cell/luorudy/const.py  | grep gkr_
#!cat elecpy/cell/luorudy/const.py  | grep gks_

  'gna_'  : 4         , # Max. Conductance of the Na Channel (mS/uF)
  'gacai_' : 0.01*0.68      , # Activity coefficient of Ca
  'gacao_' : 0.341*0.68     , # Activity coefficient of Ca


# シミュレーション実行

In [2]:
import copy
import time
import os, sys
import shutil
import json
import numpy as np

import chainer
from chainer import cuda

from matplotlib import animation, rc
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import elecpy.elecpy as elp
from elecpy.util.cmap_bipolar import bipolar
from elecpy.elecpySession import ElecpySession

from IPython.display import HTML

from scipy.signal import argrelmax
from numba.decorators import autojit

%matplotlib inline


In [3]:
path_org = "/mnt/recordings/SimulationResult/20180816-1/"
path_dst = "/mnt/recordings/SimulationResult/20180816-5/"

In [4]:
# load sample parameter
with open (os.path.join(path_org, 'sim_params.json'),'r') as f:
    sim_params = json.load(f)

sim_params['restart'] = {}
sim_params['restart']['count'] = 2200
sim_params['restart']['source'] = path_org
    
# modify parameters
sim_params['time']['end'] = 800
sim_params["stimulation"] = {}
#sim_params["stimulation"]["extracellular"][0]["start"] = 0.
#sim_params["stimulation"]["extracellular"][0]["duration"] = 10.

sim_params['geometory']['height'] = 200
sim_params['geometory']['width'] = 200

sim_params['log']['path'] = path_dst
sim_params['log']['cnt'] = 1000


print json.dumps(sim_params, indent=4)


{
    "cell_type": "luorudy", 
    "log": {
        "path": "/mnt/recordings/SimulationResult/20180816-5/", 
        "cnt": 1000
    }, 
    "stimulation": {}, 
    "time": {
        "end": 800, 
        "udt": 0.001
    }, 
    "geometory": {
        "width": 200, 
        "ds": 0.015, 
        "height": 200
    }, 
    "restart": {
        "count": 2200, 
        "source": "/mnt/recordings/SimulationResult/20180816-1/"
    }
}


In [5]:
g = elp.sim_generator(sim_params)

In [6]:
os.mkdir(sim_params['log']['path'] )

In [8]:
with open(os.path.join(path_dst, 'sim_params.json'),'w') as f: json.dump(sim_params, f, indent=4)
out = []
while True:
    img = g.next()
    if img is False: break
    out.append(np.copy(img))

StopIteration: 